In [9]:
library(Seurat)
library(stringr)

In [2]:
out = "/projects/PPC/analysis/ppc_pilot/data/robjs/"

In [2]:
load(paste(out, "raw_frazer_scRNA.robj", sep = ""), verbose = T)

Loading objects:
  tiss


In [3]:
load(paste(out, "filtered_frazer_scRNA.robj", sep = ""), verbose = T)

Loading objects:
  tiss.filtered


In [4]:
load(paste(out, "filtered_adj_frazer_scRNA.robj", sep = ""), verbose = T)

Loading objects:
  tiss.filtered_adjusted


In [85]:
out = "../Figures/Manuscript_V04/Figure_S1_"

# Cryopreserved vs. Fresh DE

In [10]:
Idents(tiss.filtered) <- "batch2plot"

levels(tiss.filtered)

[1] "Fresh iPSC"             "Fresh iPSC-PPC"         "Cryopreserved iPSC-PPC"

In [26]:
ppc_tiss.filtered = subset(tiss.filtered, idents = c("Fresh iPSC-PPC", "Cryopreserved iPSC-PPC"))

In [27]:
table(ppc_tiss.filtered@meta.data$batch2plot)


Cryopreserved iPSC-PPC         Fresh iPSC-PPC 
                 18641                  57987 

In [28]:
levels(ppc_tiss.filtered)

[1] "Fresh iPSC-PPC"         "Cryopreserved iPSC-PPC"

In [30]:
ppc_tiss.filtered = subset(ppc_tiss.filtered, cells = colnames(x = ppc_tiss.filtered)[sample(nrow(ppc_tiss.filtered@meta.data), 15000)])

In [31]:
cluster.averages <- AverageExpression(ppc_tiss.filtered)

Finished averaging RNA for cluster Fresh iPSC-PPC
Finished averaging RNA for cluster Cryopreserved iPSC-PPC


In [42]:
library(scales)

In [81]:
PlotCTCorrelation <- function(s1, s2){


    i.s <- smoothScatter(log(cluster.averages[["RNA"]][, s1], 2), log(cluster.averages[["RNA"]][, s2], 2), pch = 16, las =1 ,
        main = "", nrpoints = 10, ret.selection=TRUE, xlab = s1, ylab = s2)

    ## label the 10 very lowest-density points,the "outliers" (with obs.number):
    text(log(cluster.averages[["RNA"]][i.s[1:10],][, s1], 2), log(cluster.averages[["RNA"]][i.s[1:10],][, s2], 2), labels = rownames(cluster.averages[["RNA"]][i.s[1:10],]), cex= 0.75,
        pos = 1)
    
    
    abline(lm(cluster.averages[["RNA"]][, s2] ~ cluster.averages[["RNA"]][, s1]), col = "red")
    
    x = log(cluster.averages[["RNA"]][, s1], 2)
    x = x[! x %in% c(-Inf, Inf)]
    x2use = min(x)

    y = log(cluster.averages[["RNA"]][, s2], 2)
    y = x[! x %in% c(-Inf, Inf)]
    y2use = max(x)
    
    
    
    text(x = x2use,#min(log(cluster.averages[["RNA"]][, s1], 2)),
         y = y2use - (y2use*.1), labels = paste("r", signif(cor(cluster.averages[["RNA"]][, s2], cluster.averages[["RNA"]][, s1]), digits = 2), sep = " = "), pos = 4)
    text(x = x2use,
         y = y2use - (y2use*.2), labels = paste("p", scientific(cor.test(cluster.averages[["RNA"]][, s2], cluster.averages[["RNA"]][, s1])$p.val, digits = 1000), sep = " = "), pos = 4)


    
}

#  Figure S1

In [21]:
PlotQCMets <- function(){

    plot(tiss@meta.data$nCount_RNA, tiss@meta.data$percent.mt, pch = 20, cex = .5, xlab = "nUMI", ylab = "%MT", las = 1)
    abline(h = 10, col = "red")
    plot(tiss@meta.data$nCount_RNA, tiss@meta.data$nFeature_RNA, pch = 20, cex = .5, xlab = "nUMI", ylab = "n Unique Genes", las = 1)
    abline(h = 500, col = "red")
}

In [19]:
png(paste(out, "UMAP_unfiltered_rna.png"), width = 3, height = 3, units = "in", res = 1200)

DimPlot(tiss, group.by = "nFeature_RNA_thresh", cols = c("#8f1478", "#79c98d")) + NoLegend()

dev.off()

png 
  2

In [20]:
png(paste(out, "UMAP_unfiltered_mt.png"), width = 3, height = 3, units = "in", res = 1200)

DimPlot(tiss, group.by = "percent.mt_greater", cols = c("#79c98d", "#8f1478")) + NoLegend()

dev.off()

png 
  2

In [22]:
png(paste(out, "Scatter_unfiltered_mt.png"), width = 4, height = 3, units = "in", res = 1200)
plot(tiss@meta.data$nCount_RNA, tiss@meta.data$percent.mt, pch = 20, cex = .5, xlab = "nUMI", ylab = "%MT", las = 1)
abline(h = 10, col = "red")
dev.off()

png(paste(out, "Scatter_unfiltered_genes.png"), width = 4, height = 3, units = "in", res = 1200)
plot(tiss@meta.data$nCount_RNA, tiss@meta.data$nFeature_RNA, pch = 20, cex = .5, xlab = "nUMI", ylab = "n Unique Genes", las = 1)
abline(h = 500, col = "red")
dev.off()

png 
  2

png 
  2

In [23]:
png(paste(out, "UMAP_filtered_batch.png"), width = 3, height = 3, units = "in", res = 1200)
DimPlot(tiss.filtered, pt.size = .25, label = F, group.by = "batch2plot") + NoLegend()
dev.off()

png 
  2

In [87]:
pdf(paste(out, "smoothscatter_filtered_ff_cor.pdf"), width = 6, height = 6)
PlotCTCorrelation("Fresh iPSC-PPC", "Cryopreserved iPSC-PPC")
dev.off()

png 
  2

In [24]:
png(paste(out, "UMAP_filtered_cc.png"), width = 3, height = 3, units = "in", res = 1200)
DimPlot(tiss.filtered, group.by = "Phase", pt.size = .25) + NoLegend()
dev.off()

png 
  2

In [25]:
png(paste(out, "UMAP_filteredadj_cc.png"), width = 3, height = 3, units = "in", res = 1200)
DimPlot(tiss.filtered_adjusted, group.by = "Phase", pt.size = .25) + NoLegend()
dev.off()

png(paste(out, "UMAP_filteredadj_batch.png"), width = 3, height = 3, units = "in", res = 1200)
DimPlot(tiss.filtered_adjusted, pt.size = .25, label = F, group.by = "batch2plot") + NoLegend()
dev.off()

png 
  2

png 
  2